In [3]:
# IMPORTS _______________________________________

import json
from datetime import datetime
import matplotlib.pyplot as plt
import PySimpleGUI as sg

# Tarefas LOGIC ______________________________

def load_tasks():
    try:
        with open('BD-tarefas.json', 'r') as file:
            tasks = json.load(file)
    except FileNotFoundError:
        tasks = []
    return tasks

def save_tasks(tasks):
    with open('BD-tarefas.json', 'w') as file:
        json.dump(tasks, file, indent=2)
        
# Função para obter uma tarefa pelo ID
def get_task_by_id(tasks, task_id):
    for task in tasks:
        if task["id"] == task_id:
            return task
    return None

# Função para criar uma nova tarefa
def create_task(tasks, title, description, due_date, priority, category):
    try:
        due_date_obj = datetime.strptime(due_date, '%Y-%m-%d')
    except ValueError:
        print("Invalid date format. Please use YYYY-MM-DD.")
        return

    task_data = {
        'id': len(tasks) + 1,
        'titulo': title,
        'descricao': description,
        'data_vencimento': due_date_obj.strftime('%Y-%m-%d'),
        'prioridade': priority,
        'categoria': category,
        'concluida': False
    }
    tasks.append(task_data)
    save_tasks(tasks)
    print("Tarefa criada com sucesso.")

# Função para atualizar os detalhes de uma tarefa
def update_task(tasks, task_id, due_date=None, description=None, priority=None, category=None):
    for task in tasks:
        if task['id'] == task_id:
            if due_date is not None:
                task['data_vencimento'] = due_date
            if description is not None:
                task['descricao'] = description
            if priority is not None:
                task['prioridade'] = priority
            if category is not None:
                task['categoria'] = category
            save_tasks(tasks)
            print("Tarefa atualizada com sucesso.")
        else:
            print(f"Nenhuma tarefa encontrada com este ID {task_id}.")

#Função para organizar tarefas em diferentes categorias e visualizar as tarefas por categoria
def organize_tasks_by_category():
    categories = set(task['categoria'] for task in tasks)
    organized_tasks = {category: [task for task in tasks if task['categoria'] == category] for category in categories}
    return organized_tasks
        
# Função para priorizar as tarefas com base na prioridade
def prioritize_tasks(tasks):
    tasks.sort(key=lambda x: x['prioridade'])

# Função para gerar um relatório de progresso
def generate_progress_report(tasks):
    completed_tasks = [task for task in tasks if task['concluida']]
    pending_tasks = [task for task in tasks if not task['concluida']]

    report = {
        "total_tasks": len(tasks),
        "completed_tasks": len(completed_tasks),
        "pending_tasks": len(pending_tasks),
        "completed_task_titles": [task["titulo"] for task in completed_tasks],
        "pending_task_titles": [task["titulo"] for task in pending_tasks]
    }

    print("Relatório de progresso:")
    print(f"Total de Tarefas: {report['total_tasks']}")
    print(f"Tarefas Concluidas: {report['completed_tasks']}")
    print(f"Tarefas Pendentes: {report['pending_tasks']}")
    print("Título de Tarefas Concluidas:")
    for title in report['completed_task_titles']:
        print(f"  - {title}")
    print("Título de Tarefas Pendentes:")
    for title in report['pending_task_titles']:
        print(f"  - {title}")
        
# Função para gerar um painel com a distribuição das tarefas por mês
def generate_dashboard(tasks):
    due_dates = [datetime.strptime(task["data_vencimento"], '%Y-%m-%d') for task in tasks]
    months = [date.strftime('%b') for date in due_dates]
    
    # Distribuição de Tarefas por Categorias
    organized_tasks = organize_tasks_by_category()
    categories = list(organized_tasks.keys())
    task_counts_by_category = [len(organized_tasks[category]) for category in categories]

    # Task distribution by priority
    prioritize_tasks()
    task_counts_by_priority = [task["prioridade"] for task in tasks]

     # Task distribution by months
    task_counts_by_month = {month: months.count(month) for month in set(months)}
    
    # Plotting dashboards
    plt.figure(figsize=(10, 6))
    
    plt.subplot(1, 3, 1)
    plt.bar(categories, task_counts_by_category, color='skyblue')
    plt.title('Distribuição de Tarefas por Mês')
    plt.xlabel('Categoria')
    plt.ylabel('Número de Tarefas')

    plt.subplot(1, 3, 2)
    plt.hist(task_counts_by_priority, bins=5, edgecolor='black', alpha=0.7)
    plt.title('Distribuição de Tarefas por Mês')
    plt.xlabel('Prioridade')
    plt.ylabel('Número de Tarefas')

    plt.subplot(1, 3, 3)
    plt.bar(set(months), task_counts_by_month, color='lightcoral')
    plt.title('Distribuição de Tarefas por Mês')
    plt.xlabel('Mês')
    plt.ylabel('Número de Tarefas')

    plt.tight_layout()
    plt.show()

# Função para exibir a mensagem de ajuda com os comandos disponíveis
def print_help():
    print("Comandos disponíveis:")
    print("  help: Exibir esta mensagem de ajuda")
    print("  create: Criar uma nova tarefa")
    print("  update <task_id>: Atualizar detalhes da tarefa")
    print("  view <task_id>: Visualizar detalhes da tarefa")
    print("  list <sort_by>: Listar tarefas (opcional: sort_by due_date, priority, category)")
    print("  organize: Organizar tarefas por categoria")
    print("  prioritize: Priorizar tarefas")
    print("  report: Gerar relatório de progresso")
    print("  dashboard: Gerar e exibir um painel")
    print("  delete <task_id>: Excluir uma tarefa")
    print("  exit: Sair do programa")

# Inicializar as tarefas a partir do arquivo JSON
if __name__ == "__main__":
    tasks = load_tasks()

    while True:
        # Obter comando do usuário
        command = input("Digite um comando (digite 'help' para ver a lista de comandos): ").split()
        action = command[0]

        # Executar ação com base no comando fornecido
        if action == "help":
            print_help()

        elif action == "create":
            title = input("Digite o título da tarefa: ")
            description = input("Digite a descrição da tarefa: ")
            due_date = input("Digite a data de vencimento (AAAA-MM-DD): ")
            priority = input("Digite a prioridade: ")
            category = input("Digite a categoria: ")
            create_task(tasks, title, description, due_date, priority, category)

        elif action == "update":
            task_id = int(command[1])
            due_date = input("Digite a nova data de vencimento (AAAA-MM-DD, pressione Enter para pular): ")
            description = input("Digite a nova descrição (pressione Enter para pular): ")
            priority = input("Digite a nova prioridade (pressione Enter para pular): ")
            category = input("Digite a nova categoria (pressione Enter para pular): ")
            update_task(tasks, task_id, due_date, description, priority, category)

        elif action == "view":
            task_id = float(int(command[1]))
            task = get_task_by_id(tasks, task_id)
            if task:
                print(f"Detalhes da Tarefa {task['id']}:")
                print(f"  Título: {task['titulo']}")
                print(f"  Descrição: {task['descricao']}")
                print(f"  Data de Vencimento: {task['data_vencimento']}")
                print(f"  Prioridade: {task['prioridade']}")
                print(f"  Categoria: {task['categoria']}")
                print(f"  Concluída: {task['concluida']}")
            else:
                print(f"Nenhuma tarefa encontrada com o ID {task_id}")

        elif action == "list":
            sort_by = command[1] if len(command) > 1 else None
            valid_sort_fields = ["due_date", "priority", "category"]

            # Fornecendo um valor padrão para classificação
            default_sort_value = ""  # Ou qualquer valor padrão que faça sentido para sua aplicação
            if sort_by in valid_sort_fields:
                sorted_tasks = sorted(tasks_with_default, key=lambda x: x.get(sort_by, default_sort_value))
                for task in sorted_tasks:
                    print(f"Tarefa {task['id']}: {task['titulo']} - Data de Vencimento: {task['data_vencimento']}, Prioridade: {task['prioridade']}, Categoria: {task['categoria']}")
            else:
                for task in tasks_with_default:
                    print(f"Tarefa {task['id']}: {task['titulo']} - Data de Vencimento: {task['data_vencimento']}, Prioridade: {task['prioridade']}, Categoria: {task['categoria']}")

        elif action == "organize":
            organized_tasks = organize_tasks_by_category(tasks)
            for category, tasks in organized_tasks.items():
                print(f"Categoria: {category}")
                for task in tasks:
                    print(f"  Tarefa {task['id']}: {task['titulo']} - Data de Vencimento: {task['data_vencimento']}, Prioridade: {task['prioridade']}")

        elif action == "prioritize":
            prioritize_tasks(tasks)
            print("Tarefas priorizadas com sucesso.")

        elif action == "report":
            progress_report = generate_progress_report(tasks)
            print("Relatório de Progresso:")
            print(f"  Total de Tarefas: {progress_report['total_tasks']}")
            print(f"  Tarefas Concluídas: {progress_report['completed_tasks']}")
            print(f"  Tarefas Pendentes: {progress_report['pending_tasks']}")
            print("Títulos de Tarefas Concluídas:")
            for title in progress_report['completed_task_titles']:
                print(f"  - {title}")
            print("Títulos de Tarefas Pendentes:")
            for title in progress_report['pending_task_titles']:
                print(f"  - {title}")

        elif action == "dashboard":
            generate_dashboard(tasks)

        elif action == "delete":
            task_id = int(command[1])
            tasks = [task for task in tasks if task['id'] != task_id]
            save_tasks(tasks)
            print(f"Tarefa {task_id} excluída com sucesso.")

        elif action == "exit":
            break

        else:
            print("Comando inválido. Digite 'help' para ver a lista de comandos.")

    input("Pressione Enter para sair...")

# Retorna tarefas para a GUI
def gui_get_tasks():
    gui_tasks = []
    for task in tasks_data:
        gui_tasks.append([
            task['id'],
            task['titulo'],
            task['descricao'],
            ', '.join(task['data_vencimento']),  
            task['prioridade'],
            task['categoria']
        ])
    return sorted(gui_tasks, key=lambda x: x[0])  # Ordenado pelas IDs das tarefas

# Retorna tarefas para a GUI (filtradas por ID)
def gui_get_tasks_id(task_id):
    filtered_task = search_id(int(task_id))
    if filtered_task:
        return [[
            filtered_task['id'],
            filtered_task['titulo'],
            filtered_task['descricao'],
            ', '.join(filtered_task['data_vencimento']),
            filtered_task['prioridade'],
            filtered_task['categoria']
        ]]
    else:
        return [['', '', '', '', '', '']]  # Retorna uma lista vazia se não encontrar a tarefa

# Retorna tarefas para a GUI (filtradas por título)
def gui_get_tasks_title(task_title):
    filtered_tasks = search_title(task_title)
    gui_tasks = []
    for task in filtered_tasks:
        gui_tasks.append([
            task['id'],
            task['titulo'],
            task['descricao'],
            ', '.join(task['data_vencimento']),
            task['prioridade'],
            task['categoria']
        ])
    return sorted(gui_tasks, key=lambda x: x[0])  # Ordenado pelas IDs das tarefas


# ...............................................

tasks_data_file_path = './BD-tarefas.json'
tasks_data = load_tasks()  # Imports tasks data

# GUI ___________________________________________
sg.theme('DarkBlue3')


search_input_column = [[sg.Combo(['id', 'titulo', 'descricao', 'data_vencimento', 'prioridade', 'categoria'], key='search_input_label', size=(5, 1)),sg.Input(key='search_input', size=(35, 1)),
                        sg.Button('>', key='search_button', size=(2, 1))]]

def get_window_main():
    add_tasks_column = [[sg.Text(
        'ADD TASKS (use commas to separate multiple values)', size=(42, 1))],
        [sg.Text('titulo:', size=(7, 1)), sg.Input(
            key='add_tasks_input_title', size=(38, 1))],
        [sg.Text('descricao:', size=(7, 1)), sg.Input(
            key='add_task_input_description', size=(38, 1))],
        [sg.Text('data_vencimento:', size=(7, 1)), sg.Input(
            key='add_task_input_due_date', size=(38, 1))],
        [sg.Text('prioridade:', size=(7, 1)), sg.Input(
            key='add_task_input_priority', size=(38, 1))],
        [sg.Text('categoria:', size=(7, 1)), sg.Input(
            key='add_task_input_category', size=(38, 1))],
        [sg.Button('>', key='add_task_button', size=(2, 1))]]


# Criando a janela principal
    
    layout_main = [[sg.Text((str(get_tasks_number()) + ' tasks'), key='task_id')],
                   [sg.Button('Show all tasks', key='show_all', size=(43, 1))],
                   [sg.Text('')],
                   [sg.Column(add_tasks_column, key='add_tasks_column')],
                   [sg.Text('')],
                   [sg.Text('SEARCH BY...', size=(42, 1))],
                   [sg.Column(search_input_column, key='search_input_column')],
                   [sg.Text('')],
                   [sg.Text('STATISTICS...', size=(42, 1))],
                   [sg.Button('Tarefas concluidas', key='completed_tasks', size=(9, 1)),
                    sg.Button('categoria', key='list_category', size=(9, 1))],
                   [sg.Text('')],
                   [sg.Button('Save', key='save_changes', size=(9, 1)),
                    sg.Button('Exit', key='exit', size=(9, 1))]]

    return (sg.Window('Tarefas', layout_main, finalize=True))

def get_window_table_tasks(table_data):
    layout_table_tasks = [[sg.Table(key='content_table_tasks', visible=True, headings=['Id', 'Title', 'Description', 'due_date', 'Priority','Category'], values=table_data,
                                     max_col_width=40, auto_size_columns=True, display_row_numbers=False, justification='left', num_rows=13, row_height=35)]]
    return (sg.Window('Tasks', layout_table_tasks, finalize=True))


def get_window_table_completed_tasks(table_data):
    layout_table_completed_tasks = [[sg.Table(key='content_table_completed_tasks', visible=True, headings=['due_date', 'Id'], values=table_data, max_col_width=40,
                                     auto_size_columns=True, display_row_numbers=False, justification='left', num_rows=13, row_height=35)]]
    return (sg.Window('Completed tasks', layout_table_completed_tasks, finalize=True))


def get_window_table_category(table_data):
    layout_table_category = [[sg.Table(key='content_table_category', visible=True, headings=['category', 'title'], values=table_data, max_col_width=40,
                                     auto_size_columns=True, display_row_numbers=False, justification='left', num_rows=13, row_height=35)]]
    return (sg.Window('category', layout_table_category, finalize=True))


def update_task_number():
    window_main['task_number'].Update((str(get_task_number()) + ' task'))


window_main = get_window_main()

window_main = get_window_main()

while True:
    event, values = window_main.Read()

    if event == 'show_all':
        get_window_table_tasks(gui_get_tasks())

    elif event == 'add_task_button':
        title = values['add_task_input_title']
        description = values['add_task_input_description']
        due_date = values['add_task_input_due_date'].replace(', ', ',')
        priority = values['add_task_input_priority'].replace(', ', ',')
        category = values['add_task_input_category'].replace(', ', ',')

        if title and description and due_date and priority and category:
            create_task(tasks, title, description, due_date, priority, category)
            update_task_number()

            window_main['add_task_input_title'].Update('')
            window_main['add_task_input_description'].Update('')
            window_main['add_task_input_due_date'].Update('')
            window_main['add_task_input_priority'].Update('')
            window_main['add_task_input_category'].Update('')

        else:
            print('Missing values')

    elif event == 'search_button':
        label = values['search_input_label']
        input_value = values['search_input']

        if input_value:
            if label == 'id':
                get_window_table_tasks(gui_get_tasks_id(values['search_input']))

            elif label == 'titulo':
                get_window_table_tasks(gui_get_tasks_title(values['search_input']))

            # Add other cases for different labels...

            else:
                print('Empty filter')
        else:
            print('Filter input is empty')
            
    elif event == 'list_completed_tasks':
        print('Show completed tasks stats')
        get_window_table_completed_tasks(gui_get_due_date_dist())

    elif event == 'list_category':
        print('Show genres stats...')
        get_window_table_tasks(gui_get_category_dist())

    elif event == 'save_changes':
        print('Save changes...')
        export_json_file()

    elif event == sg.WIN_CLOSED or event == 'exit':
        break


Comandos disponíveis:
  help: Exibir esta mensagem de ajuda
  create: Criar uma nova tarefa
  update <task_id>: Atualizar detalhes da tarefa
  view <task_id>: Visualizar detalhes da tarefa
  list <sort_by>: Listar tarefas (opcional: sort_by due_date, priority, category)
  organize: Organizar tarefas por categoria
  prioritize: Priorizar tarefas
  report: Gerar relatório de progresso
  dashboard: Gerar e exibir um painel
  delete <task_id>: Excluir uma tarefa
  exit: Sair do programa
Tarefa criada com sucesso.


IndexError: list index out of range